In [1]:
import ee
import geemap

In [2]:
import ee
service_account = 'r2d2-eihwbo@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
ee.Initialize(credentials)

In [3]:
collection = ee.ImageCollection("COPERNICUS/S2_SR")
#geometry = ee.Geometry.Point(80.15569166816407,12.889524134340943)

geometry = ee.Geometry.Polygon([[[76.9159401,10.9925428],
                                 [76.9159852,10.9918602],
                                 [76.9153301,10.9917714],
                                 [76.9152366,10.9924804],
                                 [76.9159401,10.9925428]]])
geometry1 = ee.Geometry.Polygon(
        [[[76.92366482419152, 11.001849863733986],
          [76.92441584271569, 11.001934116930132],
          [76.92440084322088, 11.001233761505214],
          [76.92499355658755, 11.00097842214154],
          [76.92574457511172, 11.000883636986831],
          [76.9263603793134, 11.000953718315454],
          [76.92682171926396, 11.00092475618566],
          [76.92795897588627, 11.000922399430593],
          [76.92884410486118, 11.00119095725443],
          [76.92859401133705, 11.003214509894416],
          [76.92837407019783, 11.004215010409895],
          [76.92773034003426, 11.004183415708717],
          [76.92657162573983, 11.00409916315569],
          [76.92539145377327, 11.003941189553833],
          [76.92447413829018, 11.003777950076262],
          [76.92335833933998, 11.003567318358687]]]);
collection = collection.filterBounds(geometry).filterDate('2021-02-01','2021-04-30')
geometry2 = [[[76.9159401,10.9925428],[76.9159852,10.9918602],[76.9153301,10.9917714],[76.9152366,10.9924804],[76.9159401,10.9925428]]]

In [4]:
print(collection.size().getInfo())

36


In [5]:
print(type(collection))

<class 'ee.imagecollection.ImageCollection'>


In [6]:
S2CLD = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")


In [7]:
def clipfunc(image):
    return ee.Image(image).clip(geometry);

collection = collection.map(clipfunc)


In [8]:
def setCloudPercent(s2L2AImage):
    startDt = ee.Image(s2L2AImage).date()
    deltaMinusHour = startDt.advance(-1.0,'hour')
    deltaPlusHour = startDt.advance(1.0,'hour')
    imgColl = S2CLD.filterDate(deltaMinusHour,deltaPlusHour).filterBounds(geometry)
    img = imgColl.first()
    imgId = img.get('system:index')
    mean = ee.Dictionary(img.reduceRegion(**{
        'reducer':ee.Reducer.mean(), 
        'geometry': geometry,
        'scale': 10}))
    cloudScore = mean.get('probability')
    return s2L2AImage.set({'cloud_score':cloudScore}).set({'S2CLDImageId':imgId})

In [9]:
collection = collection.map(setCloudPercent)
collection = collection.filter(ee.Filter.lt("cloud_score", 40))

In [10]:
print(collection.size().getInfo())

20


In [11]:
def setDate(image):
    dt = ee.Image(image).date()
    dtStr = dt.format("dd-MM-YYYY")
    return image.set({'date':dtStr})

def removeDuplicateDailyImgId(date):
    return imgCollDailyDate.filter(ee.Filter.eq("date", ee.String(date))).first()


imgCollDailyDate = collection.map(setDate)
imgCollUniqueDt = imgCollDailyDate.aggregate_array('date').distinct()
collection = ee.ImageCollection(imgCollUniqueDt.map(removeDuplicateDailyImgId))

In [12]:
print(collection.size().getInfo())


10


In [13]:
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())


In [14]:
dates  = ymdList(collection)

In [15]:
dates

['20210202',
 '20210207',
 '20210212',
 '20210227',
 '20210314',
 '20210319',
 '20210324',
 '20210403',
 '20210413',
 '20210428']

In [16]:
from datetime import date as datet
mods = []
for i in range (0,len(dates)-1):
    adate = dates[i]
    bdate = dates[i+1]
    d0 = datet(int(adate[0:4]), int(adate[4:6]), int(adate[6:]))
    d1 = datet(int(bdate[0:4]), int(bdate[4:6]), int(bdate[6:]))
    delta = d1 - d0
    mod = (delta.days)//5
    mods.append(mod)

In [17]:
mods

[1, 1, 3, 3, 1, 1, 2, 2, 3]

In [18]:
factor = 0
factors = []
for mod in mods:
    factor = factor + mod
    factors.append(factor)
        

In [19]:
factors

[1, 2, 5, 8, 9, 10, 12, 14, 17]

In [20]:
def areaEqualizer(image):
    area = geometry.area(5);
    clipped = ee.Image(image).clip(geometry).geometry().area(5);
    area_equ = ee.Number(100).subtract(ee.Number(area.subtract(clipped)).divide(area).multiply(ee.Number(100)));
    return image.set({"area_score":area_equ})

In [21]:
collection = collection.map(areaEqualizer)

In [22]:
collection = collection.filter(ee.Filter.eq("area_score", ee.Number(100)))


In [23]:
print(collection.size().getInfo())

10


In [24]:
def addNDVI100(image):
    ndvi = (image.expression("((b8 - b4) / (b8 + b4)) * (100)",{
    "b8": image.select("B8"),
    "b4": image.select("B4")
    })).toInt().rename('NDVI100')
    return image.addBands(ndvi) 



collection = collection.map(addNDVI100)
collection100 = collection.select('NDVI100')

first = collection100.first()


bandNames = first.bandNames()
print(bandNames.getInfo())

['NDVI100']


In [25]:
print(collection100.size().getInfo())

10


In [26]:
collList = collection100.toList(collection100.size())
img1 = ee.Image()


In [27]:
print(collection.size().getInfo())

10


In [28]:
def calcFImage(img1,img2,fmult):
    fmult = ee.Number(fmult)
    img1 = ee.Image(img1)
    img2 = ee.Image(img2)
    eImg = img1.eq(img2)
    rcImg = img1.subtract(img2)
    collectionFromImages = ee.ImageCollection.fromImages([img1, img2])
    maxImg = collectionFromImages.select('NDVI100').max()
    lImg = maxImg.neq(img2)
    lrc = rcImg.multiply(lImg)
    lrc = lrc.add(eImg)
    constImg = ee.Image(fmult).rename("NDVI100").clip(geometry)
    mlrc = lrc.multiply(constImg).toInt()
    return mlrc
    
    
def calcMean(zip_list):
    zip_list = ee.List(zip_list) 
    fmult = zip_list.get(1)
    i = ee.Number(zip_list.get(0))
    firsti = i.subtract(1)
    img1=ee.Image(collList.get(firsti)) 
    img2=ee.Image(collList.get(i))
    binImage = calcFImage(img1,img2,fmult)
    return binImage



    

def calc(i):
    i = ee.Number(i)
    firsti = i.subtract(1)
    img1=ee.Image(collList.get(firsti)) 
    img2=ee.Image(collList.get(i))
    date1 = ee.Date(img1.date().format("YYYY-MM-dd"))
    date2 = ee.Date(img2.date().format("YYYY-MM-dd"))
    diff = ee.Number(date2.difference(date1,'day'))
    mod = diff.divide(5)
    #diff = abs(date2-date1.days)
    """d1 = date(int(adate[0:4]), int(adate[4:6]), int(adate[6:]))
    d2 = date(int(bdate[0:4]), int(bdate[4:6]), int(bdate[6:]))
    delta = d2 - d1"""
    return mod


In [29]:
finImgCollList = ee.List.sequence(1,collList.size().subtract(1),1).map(calc)


In [30]:
print(finImgCollList.getInfo())

[1, 1, 3, 3, 1, 1, 2, 2, 3]


In [31]:
"""finImgCollList = ee.List.sequence(1,collList.size().subtract(1),1).map(calcMean)
collfin = ee.ImageCollection(finImgCollList)"""

'finImgCollList = ee.List.sequence(1,collList.size().subtract(1),1).map(calcMean)\ncollfin = ee.ImageCollection(finImgCollList)'

In [32]:
x = ee.List.sequence(1,collList.size().subtract(1),1)
seq = list(range(1,collList.size().getInfo()))
merged_list = ee.List(tuple(zip(seq, factors))) 
finImgCollList = merged_list.map(calcMean)

In [33]:
#gg = ee.List.sequence(1,collList.size().subtract(1),1).map(calc)

In [34]:
collfin = ee.ImageCollection(finImgCollList)

In [35]:
print(collfin.size().getInfo())

9


In [36]:
finalImg = collfin.sum()

In [37]:
minNdvi = ee.Number(finalImg.select("NDVI100").reduceRegion(**{
     'reducer': ee.Reducer.min(),
    'geometry':geometry,
     'scale': 10,
     'maxPixels': 1e9
     }).values().get(0));
finalImg=finalImg.set({"minNdvi":minNdvi})

maxNdvi = ee.Number(finalImg.select("NDVI100").reduceRegion(**{
     'reducer': ee.Reducer.max(),
    'geometry':geometry,
     'scale': 10,
     'maxPixels': 1e9
     }).values().get(0));
finalImg=finalImg.set({"maxNdvi":maxNdvi})

In [38]:
print(minNdvi.getInfo())
print(maxNdvi.getInfo())

129
394


In [39]:
def addnorm(image):
    norm = (image.expression("((ndvi-min)/(max-min))*10",{
    "ndvi": image.select("NDVI100"),
    "min":minNdvi,
    "max":maxNdvi
    })).toInt().rename('norm')
    return image.addBands(norm) 
finalImg = addnorm(finalImg)


"""norm =(((finalImg.select("NDVI100").subtract(minNdvi))).divide((maxNdvi.subtract(minNdvi)))).rename('norm')
finalImg = finalImg.addBands(norm)"""

'norm =(((finalImg.select("NDVI100").subtract(minNdvi))).divide((maxNdvi.subtract(minNdvi)))).rename(\'norm\')\nfinalImg = finalImg.addBands(norm)'

In [40]:
const = ee.Image(5).rename("NDVI100").clip(geometry)
print(const.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI100', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 5, 'max': 5}, 'dimensions': [1, 1], 'origin': [76, 10], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[76.9159401, 10.9925428], [76.9152366, 10.9924804], [76.9153301, 10.9917714], [76.9159852, 10.9918602], [76.9159401, 10.9925428]]]}}}


In [41]:
print(finalImg.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI100', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'norm', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'maxNdvi': 394, 'minNdvi': 129}}


In [42]:
finalImg = finalImg.clip(geometry)
finalImgnd = finalImg.select("NDVI100")
finalImg = finalImg.select("norm")


In [43]:
print(finalImgnd.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI100', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'dimensions': [1, 1], 'origin': [76, 10], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'maxNdvi': 394, 'system:footprint': {'type': 'Polygon', 'coordinates': [[[76.9159401, 10.9925428], [76.9152366, 10.9924804], [76.9153301, 10.9917714], [76.9159852, 10.9918602], [76.9159401, 10.9925428]]]}, 'minNdvi': 129}}


In [44]:
finalImg1 = finalImg.select("norm")

In [45]:
finalImg1 = finalImg1.focal_median(1, 'square')

In [46]:
vector = finalImg.toInt().reduceToVectors(scale = 10)

In [47]:
def setArea(feature):
    feature=ee.Feature(feature)
    area=feature.area(5)
    feature =feature.set("area",area)
    return feature
arVector = vector.map(setArea)

In [48]:
print(arVector.getInfo())

{'type': 'FeatureCollection', 'columns': {'area': 'Float', 'count': 'Long<0, 4294967295>', 'label': 'Integer', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[76.91528176229643, 10.991785816486464], [76.91537159382484, 10.991785816486464], [76.91537159382484, 10.991965479543289], [76.91528176229643, 10.991965479543289], [76.91528176229643, 10.991785816486464]]]}, 'id': '+856217+122361', 'properties': {'area': 195.89215324580599, 'count': 2, 'label': 4}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[76.91528176229643, 10.991965479543289], [76.91537159382484, 10.991965479543289], [76.91537159382484, 10.9920553110717], [76.91528176229643, 10.9920553110717], [76.91528176229643, 10.991965479543289]]]}, 'id': '+856217+122362', 'properties': {'area': 97.94603187963067, 'count': 1, 'label': 5}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coor

In [49]:
taskConfig = {
                "folder": "stress_csv",
                "description": "hello",
                "collection": arVector,
                "fileNamePrefix": '2021-04-29',
                "fileFormat": 'CSV'
                
            }
task = ee.batch.Export.table.toDrive(**taskConfig)
task.start()

In [55]:
task.status()

{'state': 'COMPLETED',
 'description': 'hello',
 'creation_timestamp_ms': 1637441363095,
 'update_timestamp_ms': 1637441382821,
 'start_timestamp_ms': 1637441373837,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://drive.google.com/#folders/1N1LWwQw752ma-FZDpsOi2GLFu3iu_hVN'],
 'attempt': 1,
 'id': 'LF4P7QSW6OA43IBLR2ZKYF7R',
 'name': 'projects/earthengine-legacy/operations/LF4P7QSW6OA43IBLR2ZKYF7R'}

In [54]:
Map = geemap.Map(center=[10.9925428,76.9159401], zoom=18)

#Map.addLayer(finalImg.select("NDVI100"),{}, 'finalImage100')
Map.addLayer(collection.select("NDVI100"),{'min':0,'max':10}, 'finalImagenorm')
Map.addLayer(finalImgnd,{}, 'ffin')
#Map.addLayer(const,{}, 'const')
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(center=[10.9925428, 76.9159401], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

In [52]:
for i in range (1,10):
    for j in range (1,10):
        if j%4==0:
            break
        else:
            print(j)

1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3


In [21]:
Map = geemap.Map(center=[10.9925428,76.9159401], zoom=18)